## Import


In [ ]:
%load_ext autoreload
%autoreload 2
from model import train_garch_x, forecast_garch_x, load_features, evaluate_models
from feature_engineering import engineer_features
from data_ingestion import get_data
import pandas as pd
import numpy as np
from config import VOL_WINDOW, VOL_TARGET_HORIZON, DATA_RAW, TEST_SIZE,RESULTS_DIR
import matplotlib.pyplot as plt
from data_ingestion import get_sentiment_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data and feature engineering

In [14]:
df = get_data(live_update=True)
df['date'] = pd.to_datetime(df.index).date
sentiment_df = get_sentiment_data()

Loading historical from C:\Users\hp\Financial-volatility-forecasting\data\raw\SPY_historical.csv
✅ Full data: 8302 days (historical + 60d live)
Sentiment saved: 3 days


In [16]:
df = load_features()
returns = df["log_return"]
target = df["target_volatility"]
sentiment = df["sentiment_lag1"]

In [17]:
print(f"Engineered data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Engineered data shape: (8261, 18)
Columns: ['Close', 'High', 'Low', 'Open', 'Volume', 'log_return', 'volatility_20d', 'abs_return', 'return_squared', 'lag_1', 'lag_5', 'lag_10', 'lag_20', 'rolling_abs_return_mean_20d', 'sentiment', 'sentiment_lag1', 'sentiment_scaled', 'target_volatility']


## Load sentiment feature

In [20]:
split_idx = int(len(df) * (1 - TEST_SIZE))


sentiment = df["sentiment_lag1"]
sentiment_train = sentiment.iloc[:split_idx]
sentiment_test = sentiment.iloc[split_idx:]
returns_train = returns.iloc[:split_idx]
returns_test = returns.iloc[split_idx:]
target_test = target.iloc[split_idx:]



In [21]:
print(f"\nTrain size: {len(returns_train)}, Test size: {len(returns_test)}")



Train size: 6608, Test size: 1653


## Rolling GARCH-X loop (parallel to GARCH)

In [22]:
garch_x_preds = []

for t in range(len(returns) - split_idx):
    r_train = returns.iloc[:split_idx + t]
    x_train = sentiment.iloc[:split_idx + t].values.reshape(-1,1)
    
    model = train_garch_x(r_train, x_train)
    
    x_next = np.array([[sentiment.iloc[split_idx + t]]])

    pred = forecast_garch_x(model, x_next)
    garch_x_preds.append(pred)


C:\Users\hp\Financial-volatility-forecasting\src\model.py:134: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  res = model.fit(disp="off")
C:\Users\hp\Financial-volatility-forecasting\src\model.py:134: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  res = model.fit(disp="off")
C:\Users\hp\Financial-volatility-forecasting\src\model.py:134: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  res = model.fit(disp="off")
C:\Users\hp\Financial-volatility-forecasting\src\model.py:134: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  res = model.fit(disp="off")
C:\Users\hp\Financia

## Evaluate

In [25]:
print(model.params)

# const = Mean equation intercept
# x0 = Sentiment impact
# omega = Baseline volatility
# alpha[1] = shock persistence
# beta[1] = volatility persistence


Const       0.005156
x0          0.000000
omega       0.000003
alpha[1]    0.100002
beta[1]     0.879998
Name: params, dtype: float64


In [ ]:
metrics_x = evaluate_models(
    y_true=target_test.values, 
    y_pred=garch_x_preds, 
    model_name="GARCH-X")

## Save Predictions

In [29]:
garch_x_preds_df = pd.DataFrame({
    "target_vol": target_test.values,
    "garch_x_pred": garch_x_preds,
})
garch_x_preds_df.to_csv(RESULTS_DIR / "preds" / "garch_x_preds.csv", index=False)
print(f" Predictions saved to {RESULTS_DIR / 'preds' / 'garch_x_preds.csv'}")

 Predictions saved to C:\Users\hp\Financial-volatility-forecasting\results\preds\garch_x_preds.csv
